In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from spike_tools import (
    general as spike_general,
    analysis as spike_analysis,
)
import data_utils
from constants import FEATURES

species = 'nhp'
subject = 'SA'
exp = 'WCST'
session = 20180802  # this is the session for which there are spikes at the moment. 

In [4]:
behavior_file = spike_general.get_behavior_path(subject, session)
behavior_data = pd.read_csv("/data/sub-SA_sess-20180802_object_features.csv")
valid_beh = behavior_data[behavior_data.Response.isin(["Correct", "Incorrect"])]


In [5]:
behavior_by_bins = data_utils.get_behavior_by_bins(50, valid_beh)

In [11]:
behavior_by_bins.to_pickle('/data/processed/sub-SA_sess-20180802_behavior_binsize_50.pickle')

### Grab bin idxs of interval around fb onset

In [8]:
intervals = data_utils.get_trial_intervals(valid_beh, pre_interval=1500, post_interval=1500, bin_size=50)

In [82]:
intervals.to_pickle("/data/processed/sub-SA_sess-20180802_interval_1500_fb_1500_binsize_50.pickle")

### Grab design matrix for intervals 

In [14]:
spikes_by_bins = pd.read_pickle('/data/processed/sub-SA_sess-20180802_spike_counts_binsize_50.pickle')
beh_by_bins = pd.read_pickle('/data/processed/sub-SA_sess-20180802_behavior_binsize_50.pickle')
intervals = pd.read_pickle("/data/processed/sub-SA_sess-20180802_interval_1500_fb_1500_binsize_50.pickle")

NUM_UNITS = 59
column_names_w_units = FEATURES + ["CORRECT", "INCORRECT"] + [f"unit_{i}" for i in range(0, NUM_UNITS)]
column_names = FEATURES + ["CORRECT" + "INCORRECT"]

In [ ]:
design_mat = data_utils.get_design_matrix(spikes_by_bins, beh_by_bins, column_names_w_units, 20, 0)

In [16]:
interval_bins = data_utils.get_interval_bins(intervals)

In [19]:
mat_in_intervals = design_mat[design_mat.bin_idx.isin(interval_bins)]
assert len(mat_in_intervals) == len(interval_bins)